# POC: Busca Vetorial OTIMIZADA para DNE Real (1.5M+ registros)

## 🚀 Otimizações implementadas:

### 1. **Modelo de embedding mais leve:**
- ❌ `neuralmind/bert-base-portuguese-cased` (110M parâmetros, lento)
- ✅ `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` (118M parâmetros, **4x mais rápido**)
- ✅ Alternativa: `rufimelo/bert-large-portuguese-cased-legal-mlm-sts-v1` (otimizado para português)

"### 2. **FAISS IndexHNSWFlat ao invés de IndexFlatL2:**\n",
    "- ❌ IndexFlatL2: busca exata em TODOS os 1.5M vetores (muito lento)\n",
    "- ✅ IndexHNSWFlat: busca em grafo hierárquico (HNSW - Hierarchical Navigable Small World)\n",
    "- **Speedup: 10-50x mais rápido** com precisão ~99.5% (melhor que IVF)\n",
    "- **Sem treinamento**: apenas adiciona os vetores (mais simples)\n",

### 3. **Processamento em batches maiores:**
- Batch size aumentado de 32 para 128
- GPU automática se disponível

### 4. **Salvamento de índices:**
- Construir índices uma vez, reutilizar sempre
- Evita reprocessamento dos 1.5M registros

    "## ⏱️ Performance esperada:\n",
    "- **Construção inicial**: ~15-30min (uma única vez, sem treinamento)\n",
    "- **Busca**: <100ms por query (vs ~5s com Flat)\n",
    "- **Precisão**: ~99.5% (praticamente idêntico ao Flat)"

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import faiss
import pickle
import torch
from pathlib import Path
from typing import Dict, List, Optional
from sentence_transformers import SentenceTransformer
from unidecode import unidecode
from tqdm import tqdm
import time

# Detectar GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🔧 Usando device: {device}")

## 1. EmbeddingService OTIMIZADO

In [ ]:
class EmbeddingServiceOptimized:
    """Serviço otimizado para grandes volumes de dados"""
    
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
        """
        Modelos recomendados por velocidade:
        1. paraphrase-multilingual-MiniLM-L12-v2 (mais rápido, multilingual)
        2. all-MiniLM-L6-v2 (muito rápido, inglês mas funciona razoável em PT)
        3. rufimelo/bert-large-portuguese-cased-legal-mlm-sts-v1 (português, médio)
        """
        print(f"⚡ Carregando modelo OTIMIZADO: {model_name}")
        self.model = SentenceTransformer(model_name, device=device)
        self.embedding_dim = self.model.get_sentence_embedding_dimension()
        print(f"✅ Dimensão: {self.embedding_dim}, Device: {self.model.device}")
    
    @staticmethod
    def normalize_text(text: str) -> str:
        if not text or not isinstance(text, str):
            return ""
        
        text = unidecode(text)
        text = text.lower()
        
        replacements = {
            r'\br\.?\s': 'rua ',
            r'\bav\.?\s': 'avenida ',
            r'\btrav\.?\s': 'travessa ',
            r'\balam\.?\s': 'alameda ',
            r'\bpca\.?\s': 'praca ',
            r'\bjd\.?\s': 'jardim ',
            r'\bvl\.?\s': 'vila ',
        }
        
        for pattern, replacement in replacements.items():
            text = re.sub(pattern, replacement, text)
        
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    def embed_text(self, text: str) -> np.ndarray:
        normalized = self.normalize_text(text)
        if not normalized:
            return np.zeros(self.embedding_dim, dtype=np.float32)
        return self.model.encode(normalized, convert_to_numpy=True, show_progress_bar=False)
    
    def embed_address_fields(self, address: Dict[str, str]) -> Dict[str, np.ndarray]:
        embeddings = {}
        for field in ['logradouro', 'bairro', 'cidade']:
            text = address.get(field, '')
            embeddings[field] = self.embed_text(text)
        return embeddings
    
    def embed_batch(self, texts, batch_size: int = 128) -> np.ndarray:
        """Batch maior para melhor performance"""
        # Converter Series para lista se necessário
        if hasattr(texts, 'tolist'):
            texts = texts.tolist()
        
        normalized_texts = [self.normalize_text(t) for t in texts]
        normalized_texts = [t if t else " " for t in normalized_texts]
        
        embeddings = self.model.encode(
            normalized_texts, 
            convert_to_numpy=True, 
            show_progress_bar=True,
            batch_size=batch_size,
            device=device
        )
        return embeddings.astype(np.float32)

## 2. IndexBuilder com FAISS IVF (RÁPIDO)

In [ ]:
class IndexBuilderOptimized:
    """Construtor de índices FAISS OTIMIZADO para grandes volumes"""
    
    def __init__(self, embedding_service: EmbeddingServiceOptimized):
        self.embedding_service = embedding_service
        self.indices = {}
        self.dataframe = None
    
    "    def build_indices(self, df: pd.DataFrame, fields: list = None, use_hnsw: bool = True, M: int = 32) -> dict:\n",
    "        \"\"\"\n",
    "        Constrói índices FAISS otimizados\n",
    "        \n",
    "        Args:\n",
    "            use_hnsw: True = IndexHNSWFlat (RÁPIDO + PRECISO), False = IndexFlatL2 (lento)\n",
    "            M: Número de conexões por nó no grafo (16-64, default=32)\n",
    "                - M=16: mais rápido, menos preciso (~98%)\n",
    "                - M=32: balanceado (~99.5%)\n",
    "                - M=64: mais preciso (~99.9%), mais memória\n",
    "        \"\"\"\n",
    "        if fields is None:\n",
    "            fields = ['logradouro', 'bairro', 'cidade']\n",
    "        \n",
    "        self.dataframe = df.copy()\n",
    "        n_records = len(df)\n",
    "        \n",
    "        print(f\"🔨 Construindo índices para {n_records:,} endereços\")\n",
    "        print(f\"⚙️  Modo: {'HNSW (RÁPIDO + PRECISO)' if use_hnsw else 'Flat (LENTO)'}\")\n",
    "        \n",
    "        for field in fields:\n",
    "            print(f\"\\n📍 Processando campo: {field}\")\n",
    "            start_time = time.time()\n",
    "            \n",
    "            texts = df[field].fillna('').astype(str).tolist()\n",
    "            embeddings = self.embedding_service.embed_batch(texts, batch_size=128)\n",
    "            dimension = embeddings.shape[1]\n",
    "            \n",
    "            if use_hnsw:\n",
    "                # HNSW: grafo hierárquico navegável (sem treinamento!)\n",
    "                index = faiss.IndexHNSWFlat(dimension, M)\n",
    "                \n",
    "                # efSearch: quantos vizinhos considerar na busca\n",
    "                # Maior = mais preciso mas mais lento (default=16)\n",
    "                index.hnsw.efSearch = 32  # balanceado (16-64)\n",
    "                \n",
    "                print(f\"   🧠 Adicionando {n_records:,} vetores ao índice HNSW...\")\n",
    "                index.add(embeddings)\n",
    "            else:\n",
    "                # Flat: busca exata (lento para grandes volumes)\n",
    "                index = faiss.IndexFlatL2(dimension)\n",
    "                index.add(embeddings)\n",
    "            \n",
    "            self.indices[field] = index\n",
    "            elapsed = time.time() - start_time\n",
    "            print(f\"   ✅ Concluído em {elapsed:.1f}s\")\n",
    "        \n",
    "        return self.indices\n",
    
    def save_indices(self, output_dir: str):
        """Salva índices para reutilização"""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        print(f"💾 Salvando índices em: {output_path}")
        
        for field, index in self.indices.items():
            index_file = output_path / f"{field}_index.faiss"
            faiss.write_index(index, str(index_file))
            print(f"   ✅ {field}_index.faiss")
        
        df_file = output_path / "addresses.parquet"
        self.dataframe.to_parquet(df_file, index=False)
        print(f"   ✅ addresses.parquet")
        
        metadata = {
            'fields': list(self.indices.keys()),
            'n_records': len(self.dataframe),
            'embedding_dim': self.embedding_service.embedding_dim
        }
        metadata_file = output_path / "metadata.pkl"
        with open(metadata_file, 'wb') as f:
            pickle.dump(metadata, f)
        print(f"   ✅ metadata.pkl")
        print(f"\n🎉 Índices salvos! Use load_indices() para carregar rapidamente.")
    
    def load_indices(self, input_dir: str):
        """Carrega índices salvos (MUITO mais rápido que reconstruir)"""
        input_path = Path(input_dir)
        
        print(f"📂 Carregando índices de: {input_path}")
        start_time = time.time()
        
        metadata_file = input_path / "metadata.pkl"
        with open(metadata_file, 'rb') as f:
            metadata = pickle.load(f)
        
        df_file = input_path / "addresses.parquet"
        self.dataframe = pd.read_parquet(df_file)
        
        for field in metadata['fields']:
            index_file = input_path / f"{field}_index.faiss"
            self.indices[field] = faiss.read_index(str(index_file))
            print(f"   ✅ {field}")
        
        elapsed = time.time() - start_time
        print(f"\n⚡ Carregado em {elapsed:.1f}s ({len(self.dataframe):,} registros)")
        
        return self.indices, self.dataframe

## 3. SearchEngine (mesma lógica)

In [ ]:
class SearchEngine:
    """Motor de busca vetorial com pesos dinâmicos"""
    
    def __init__(
        self, 
        embedding_service: EmbeddingServiceOptimized,
        indices: Dict[str, faiss.Index],
        dataframe: pd.DataFrame
    ):
        self.embedding_service = embedding_service
        self.indices = indices
        self.dataframe = dataframe
        
        self.base_weights = {
            'with_cep': {
                'cep': 0.30,
                'logradouro': 0.40,
                'bairro': 0.20,
                'cidade': 0.10
            },
            'without_cep': {
                'logradouro': 0.55,
                'bairro': 0.25,
                'cidade': 0.20
            }
        }
        
        self.use_uf_filter = True
        self.confidence_threshold = 0.8
    
    def _get_dynamic_weights(self, query: Dict[str, str]) -> Dict[str, float]:
        has_cep = bool(query.get('cep'))
        weights = self.base_weights['with_cep' if has_cep else 'without_cep'].copy()
        available_fields = [f for f in ['logradouro', 'bairro', 'cidade'] if query.get(f)]
        filtered_weights = {k: v for k, v in weights.items() if k in available_fields or k == 'cep'}
        total_weight = sum(filtered_weights.values())
        if total_weight > 0:
            normalized_weights = {k: v / total_weight for k, v in filtered_weights.items()}
        else:
            normalized_weights = filtered_weights
        return normalized_weights
    
    def _calculate_field_similarity(
        self, 
        field: str, 
        query_embedding: np.ndarray, 
        top_k: int = 100
    ) -> tuple:
        index = self.indices[field]
        query_embedding = query_embedding.reshape(1, -1).astype(np.float32)
        distances, indices = index.search(query_embedding, top_k)
        similarities = 1.0 / (1.0 + distances[0])
        return similarities, indices[0]
    
    def _calculate_cep_match(self, query_cep: str, db_cep: str) -> float:
        if not query_cep or not db_cep:
            return 0.0
        
        query_clean = query_cep.replace('-', '').replace('.', '')
        db_clean = db_cep.replace('-', '').replace('.', '')
        
        if query_clean == db_clean:
            return 1.0
        
        if len(query_clean) >= 5 and len(db_clean) >= 5:
            if query_clean[:5] == db_clean[:5]:
                return 0.5
        
        return 0.0
    
    def search(
        self, 
        query: Dict[str, str], 
        top_k: int = 5,
        search_k: int = 100
    ) -> str:
        weights = self._get_dynamic_weights(query)
        query_embeddings = self.embedding_service.embed_address_fields(query)
        
        candidate_scores = {}
        field_scores_map = {}
        
        for field in ['logradouro', 'bairro', 'cidade']:
            if not query.get(field):
                continue
            
            query_emb = query_embeddings[field]
            similarities, indices = self._calculate_field_similarity(field, query_emb, search_k)
            weight = weights.get(field, 0.0)
            
            for idx, sim in zip(indices, similarities):
                if self.use_uf_filter and query.get('uf'):
                    db_uf = self.dataframe.iloc[idx]['uf']
                    if db_uf != query['uf']:
                        continue
                
                if idx not in candidate_scores:
                    candidate_scores[idx] = 0.0
                    field_scores_map[idx] = {}
                
                candidate_scores[idx] += weight * sim
                field_scores_map[idx][field] = float(sim)
        
        if query.get('cep'):
            cep_weight = weights.get('cep', 0.0)
            for idx in candidate_scores.keys():
                db_cep = self.dataframe.iloc[idx]['cep']
                cep_score = self._calculate_cep_match(query.get('cep'), db_cep)
                candidate_scores[idx] += cep_weight * cep_score
                field_scores_map[idx]['cep'] = cep_score
        
        sorted_candidates = sorted(
            candidate_scores.items(), 
            key=lambda x: x[1], 
            reverse=True
        )[:top_k]
        
        results = []
        for idx, score in sorted_candidates:
            row = self.dataframe.iloc[idx]
            
            if score >= self.confidence_threshold:
                confidence = "high"
            elif score >= 0.6:
                confidence = "medium"
            else:
                confidence = "low"
            
            result = {
                "address": {
                    "logradouro": row['logradouro'],
                    "bairro": row['bairro'],
                    "cidade": row['cidade'],
                    "uf": row['uf'],
                    "cep": row['cep']
                },
                "score": float(score),
                "confidence": confidence,
                "field_scores": field_scores_map.get(idx, {})
            }
            results.append(result)
        
        response = {
            "results": results,
            "query": query,
            "total_found": len(results),
            "weights_used": weights
        }
        
        return json.dumps(response, ensure_ascii=False, indent=2)

## 4. Pipeline: Construir Índices (EXECUTAR UMA VEZ)

In [ ]:
# Carregar DNE
dne_path = Path('../data/dne.parquet')
print(f"📂 Carregando DNE de: {dne_path}")
df_dne_raw = pd.read_parquet(dne_path)

# Mapear colunas
column_mapping = {
    'logradouro_completo': 'logradouro',
    'bairro_completo': 'bairro',
    'cidade_completo': 'cidade'
}
df_dne = df_dne_raw.rename(columns=column_mapping)

print(f"✅ Dataset: {len(df_dne):,} registros")
print(f"📊 Distribuição por UF:")
print(df_dne['uf'].value_counts().head(10))

In [ ]:
# Inicializar serviço de embeddings
embedding_service = EmbeddingServiceOptimized(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [ ]:
# Construir índices OTIMIZADOS (demora 15-30min, mas faz UMA VEZ)
index_builder = IndexBuilderOptimized(embedding_service)

# use_hnsw=True: RÁPIDO + PRECISO (99.5% recall, sem treinamento!)
# M=32: número de conexões no grafo (16=rápido, 32=balanceado, 64=preciso)
indices = index_builder.build_indices(df_dne, use_hnsw=True, M=32)

In [ ]:
# SALVAR índices para não precisar reconstruir
index_builder.save_indices('../data/indices_otimizado')

## 5. Pipeline: Carregar Índices (RÁPIDO - use sempre)

In [ ]:
# Carregar modelo
embedding_service = EmbeddingServiceOptimized(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Carregar índices salvos (segundos vs 30min!)
index_builder = IndexBuilderOptimized(embedding_service)
indices, df_dne = index_builder.load_indices('../data/indices_otimizado')

# Inicializar motor de busca
search_engine = SearchEngine(embedding_service, indices, df_dne)
print(f"\n🚀 Sistema pronto! ({len(df_dne):,} endereços indexados)")

## 6. Teste de Performance

In [ ]:
# Exemplo de busca
query = {
    'logradouro': 'Rua das Flores',
    'bairro': 'Centro',
    'cidade': 'São Paulo',
    'uf': 'SP',
    'cep': '01000-000'
}

print("⏱️  Testando performance da busca...\n")

# Medir tempo
start = time.time()
result = search_engine.search(query, top_k=5)
elapsed = time.time() - start

print(f"⚡ Busca concluída em: {elapsed*1000:.1f}ms")
print(f"\nResultados:")
print(result)

## 7. Benchmark: Múltiplas buscas

In [ ]:
# Testar 10 buscas consecutivas
n_searches = 10
times = []

print(f"🔥 Executando {n_searches} buscas...\n")

for i in range(n_searches):
    sample = df_dne.sample(1).iloc[0]
    query = {
        'logradouro': sample['logradouro'],
        'cidade': sample['cidade'],
        'uf': sample['uf']
    }
    
    start = time.time()
    result = search_engine.search(query, top_k=5)
    elapsed = time.time() - start
    times.append(elapsed)
    
    print(f"Busca {i+1}: {elapsed*1000:.1f}ms")

print(f"\n📊 Estatísticas:")
print(f"   Média: {np.mean(times)*1000:.1f}ms")
print(f"   Mediana: {np.median(times)*1000:.1f}ms")
print(f"   Min: {np.min(times)*1000:.1f}ms")
print(f"   Max: {np.max(times)*1000:.1f}ms")

## 📝 Notas de Otimização

### Para ajustar performance vs precisão:

1. **M** (conexões no grafo HNSW):
   - M=16: mais rápido, ~98% precisão
   - M=32: balanceado, ~99.5% precisão ✅
   - M=64: mais preciso, ~99.9% precisão, mais memória

2. **efSearch** (vizinhos considerados na busca):
   - `index.hnsw.efSearch = 16`: muito rápido, menos preciso
   - `index.hnsw.efSearch = 32`: balanceado ✅
   - `index.hnsw.efSearch = 64`: mais preciso, mais lento

3. **Modelo de embedding:**
   - `paraphrase-multilingual-MiniLM-L12-v2`: balanceado ✅
   - `all-MiniLM-L6-v2`: MUITO rápido
   - `neuralmind/bert-base-portuguese-cased`: mais preciso, lento

### GPU vs CPU:
- GPU: ~10x mais rápido na construção
- Busca: diferença menor (FAISS já é otimizado)

### Alternativa: FAISS GPU
```python
import faiss.contrib.torch_utils
res = faiss.StandardGpuResources()
index_gpu = faiss.index_cpu_to_gpu(res, 0, index)
```